<a href="https://colab.research.google.com/github/Ekene-Ebenezer/Gomycode-Data-Science/blob/main/Speech_recognition_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit speechrecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.5 MB/s eta 0:00:00


In [2]:
!sudo apt-get install portaudio19-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (332 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEn

In [3]:
!pip install pyaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=PyAudio-0.2.14-cp310-cp310-linux_x86_64.whl size=63862 sha256=e30f0753fef6f22f4144530d0e16d0c40129c028b4b3faf774e8636aafe0ae31
  Stored in directory: /root/.cache/pip/wheels/d6/21/f4/0b51d41ba79e51b16295cbb096ec49f334792814d545b508c5
Successfully built pyaudio


In [4]:
!pip install <'/content/PyAudio-0.2.11-cp311-cp311-win_amd64.whl'>

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `pip install <'/content/PyAudio-0.2.11-cp311-cp311-win_amd64.whl'>'


In [5]:
!pip install <'/content/PyAudio-0.2.11-cp311-cp311-win_amd64.whl'>

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `pip install <'/content/PyAudio-0.2.11-cp311-cp311-win_amd64.whl'>'


In [6]:
%%writefile app.py

import streamlit as st
import speech_recognition as sr
import os

def transcribe_speech(selected_api, language):
    # Initialize recognizer class
    r = sr.Recognizer()
    # Reading Microphone as source
    with sr.Microphone() as source:
        st.info("Speak now...")
        # listen for speech and store in audio_text variable
        audio_text = r.listen(source)
        st.info("Transcribing...")

        try:
            # using selected speech recognition API
            if selected_api == "Google":
                text = r.recognize_google(audio_text, language=language)
            elif selected_api == "Sphinx":
                text = r.recognize_sphinx(audio_text, language=language)
            return text
        except sr.UnknownValueError:
            return "Sorry, I could not understand what you said."
        except sr.RequestError as e:
            if selected_api == "Google":
                return "Sorry, Google Speech Recognition API request failed. Please check your internet connection."
            elif selected_api == "Sphinx":
                return "Sorry, Sphinx API request failed. Please ensure you have installed the necessary dependencies."
        except sr.WaitTimeoutError:
            return "Sorry, the speech recognition system timed out. Please try again."
        except Exception as e:
            return f"Sorry, an unexpected error occurred: {e}"

def save_transcription(text):
    # Open a file dialog to choose the save location
    save_location = st.file_uploader("Save Transcription As", type=["txt"])

    if save_location is not None:
        # Write the transcribed text to the file
        with open(save_location.name, "w") as file:
            file.write(text)
        st.success(f"Transcription saved successfully as '{save_location.name}'")

def main():
    st.title("Speech Recognition App")
    st.write("Click on the microphone to start speaking:")

    # Add a dropdown menu to select speech recognition API
    selected_api = st.selectbox("Select Speech Recognition API", ["Google", "Sphinx"])

    # Add a dropdown menu to select the language
    selected_language = st.selectbox("Select Language", ["English", "Spanish", "French"])

    # Map language options to language codes recognized by SpeechRecognition
    language_map = {
        "English": "en-US",
        "Spanish": "es-ES",
        "French": "fr-FR"
    }

    # Add a button to trigger speech recognition
    if st.button("Start Recording"):
        speech_started = True
        pause_button = st.button("Pause Recording")

        while speech_started:
            if pause_button:
                pause_button = st.button("Resume Recording")
                st.info("Recording paused...")
                break
            else:
                text = transcribe_speech(selected_api, language_map[selected_language])
                st.write("Transcription: ", text)

                # Add a button to save transcription to a file
                if st.button("Save Transcription"):
                    save_transcription(text)

if __name__ == "__main__":
    main()

Writing app.py


In [7]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.096s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501 & curl -s ipv4.icanhazip.com

34.85.224.230



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.85.224.230:8501

npx: installed 22 in 4.382s
your url is: https://slow-kings-lie.loca.lt
/root/.npm/_npx/2488/lib/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:43645 (check your firewall settings)
    at Socket.<anonymous> (/root/.npm/_npx/2488/lib/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (events.js:315:20)
    at emitErrorNT (internal/streams/destroy.js:106:8)
    at emitErrorCloseNT (internal/streams/destroy.js:74:3)
    at processTicksAndRejections (internal/process/task_queues.js:80:21)
